![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [162]:
# Start your code here!
import pandas as pd
import numpy as np

types = {
    'student_id': np.int32,
    'city': 'category',
    'city_development_index': np.float16,
    'gender': 'category',
    'relevant_experience': 'category', 
    'enrolled_university': 'category',
    'education_level': 'category' , #ordered
    'major_discipline': 'category',
    'experience': 'category', #ordered
    'company_size': 'category', #ordered
    'company_type': 'category',
    'last_new_job': 'category', #ordered
    'training_hours': np.int32,
    'job_change': np.int32
}

In [163]:
# Import data

ds_jobs = pd.read_csv('customer_train.csv', dtype=types)
display(ds_jobs.head())
display(ds_jobs.info())

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.775879,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624023,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789062,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767090,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  int32   
dtypes: category(10), float

None

In [164]:
ordered_columns = ['education_level', 'experience', 'company_size', 'last_new_job', 'relevant_experience', 'enrolled_university']

In [165]:
for i in ordered_columns:
    print('\n',i, '\n''\n', ds_jobs[i].value_counts().index)


 education_level 

 CategoricalIndex(['Graduate', 'Masters', 'High School', 'Phd',
                  'Primary School'],
                 categories=['Graduate', 'High School', 'Masters', 'Phd', 'Primary School'], ordered=False, dtype='category')

 experience 

 CategoricalIndex(['>20', '5', '4', '3', '6', '2', '7', '10', '9', '8', '15',
                  '11', '14', '1', '<1', '16', '12', '13', '17', '19', '18',
                  '20'],
                 categories=['1', '10', '11', '12', ..., '8', '9', '<1', '>20'], ordered=False, dtype='category')

 company_size 

 CategoricalIndex(['50-99', '100-499', '10000+', '10-49', '1000-4999', '<10',
                  '500-999', '5000-9999'],
                 categories=['10-49', '100-499', '1000-4999', '10000+', '50-99', '500-999', '5000-9999', '<10'], ordered=False, dtype='category')

 last_new_job 

 CategoricalIndex(['1', '>4', '2', 'never', '4', '3'], categories=['1', '2', '3', '4', '>4', 'never'], ordered=False, dtype='category')

 relev

In [166]:
education_level_categories=['Primary School', 'High School','Graduate', 'Masters', 'Phd']
experience_categories = ['<1', '1', '2','3','4','5','6',  '7','8','9','10','11','12','13','14','15','16','17','18','19','20','>20']
company_size_categories=['<10', '10-49', '50-99','100-499','500-999','1000-4999','5000-9999','10000+']
last_new_job_categories=['never','1', '2', '3', '4', '>4']
relevant_experience_categories=['No relevant experience', 'Has relevant experience']
enroled_categories=['no_enrollment', 'Part time course', 'Full time course']

order = [education_level_categories, experience_categories, company_size_categories, last_new_job_categories]

In [167]:
from pandas.api.types import CategoricalDtype

ds_jobs[ordered_columns[0]] = pd.Categorical( ds_jobs[ordered_columns[0]], categories=education_level_categories, ordered=True)
ds_jobs[ordered_columns[1]] = pd.Categorical(ds_jobs[ordered_columns[1]], categories=experience_categories, ordered=True)
ds_jobs[ordered_columns[2]] = pd.Categorical(ds_jobs[ordered_columns[2]], categories=company_size_categories, ordered=True)
ds_jobs[ordered_columns[3]] = pd.Categorical(ds_jobs[ordered_columns[3]], categories=last_new_job_categories, ordered=True)
ds_jobs[ordered_columns[4]] = pd.Categorical(ds_jobs[ordered_columns[4]], categories=relevant_experience_categories, ordered=True)
ds_jobs[ordered_columns[5]] = pd.Categorical(ds_jobs[ordered_columns[5]], categories=enroled_categories, ordered=True)

print(ds_jobs[ordered_columns].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   education_level      18698 non-null  category
 1   experience           19093 non-null  category
 2   company_size         13220 non-null  category
 3   last_new_job         18735 non-null  category
 4   relevant_experience  19158 non-null  category
 5   enrolled_university  18772 non-null  category
dtypes: category(6)
memory usage: 114.1 KB
None


In [168]:
# Filter to only contain students with 10 or more years of experience at companies at least 1000 employees

#ds_jobs = ds_jobs.dropna(axis=0)


In [169]:
threshold_years = 10
threshold_employees = 5

ds_jobs_clean = ds_jobs[(ds_jobs['experience'].cat.codes >= threshold_years) & (ds_jobs['company_size'].cat.codes >= threshold_employees)]
display(ds_jobs_clean.head())

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0
